In [1]:
import tensorflow as tf
import utils
import numpy as np

__author__ = "Olivares Castillo José Luis"
tf.__version__


'1.5.0-rc0'

In [2]:
es,na = utils.load_node2vec()

In [3]:
es.shape[1]

129

In [4]:
index_es, index_na = utils.get_seed_index(es, na)
es_vectores = utils.get_vectors(es, index_es)
na_vectores = utils.get_vectors(na, index_na)

In [5]:
index_es

[734,
 4199,
 3508,
 3047,
 2645,
 3174,
 1157,
 751,
 1574,
 3262,
 3421,
 1207,
 1475,
 2972,
 4312,
 1034,
 2239,
 2487,
 103,
 2939,
 2365,
 2780,
 1756,
 1745,
 602,
 3149,
 793,
 619,
 1948,
 3113,
 1352,
 3609,
 2321,
 3879,
 1359,
 1415,
 2794,
 3420,
 2923,
 3745,
 948,
 2650,
 4071,
 2347,
 1143,
 3212,
 3637,
 20,
 1973,
 3658,
 1121,
 2694,
 3683,
 1205,
 4409,
 2187,
 576,
 1146,
 2956,
 37,
 3676,
 4262,
 3442,
 3756,
 1772,
 100,
 2248,
 2336,
 1927,
 823,
 2332,
 1102,
 3962,
 443,
 3863,
 1389,
 1269,
 75,
 1641,
 1155,
 4144,
 2869,
 434,
 2242,
 3154,
 3612,
 2889,
 3519,
 2753,
 886,
 4121,
 241,
 1693,
 136,
 1206,
 4008,
 1534,
 3808,
 2886,
 1639,
 3991,
 3375,
 4098,
 1054,
 2655,
 3846,
 2801,
 4304,
 7,
 2261,
 4349,
 701,
 157,
 728,
 2767,
 2414,
 2948,
 2560,
 4329,
 761,
 1353,
 743,
 2458,
 3554,
 2392,
 4160,
 3704,
 1064,
 3799,
 3579,
 3486,
 171,
 2731,
 2734,
 3680,
 2267,
 2085,
 2368,
 3951,
 2484,
 855,
 2666,
 2453,
 1232,
 1856,
 2879,
 3985,
 2

In [6]:
lexes=[es.iloc[_][0] for _ in index_es]
lexna=[na.iloc[_][0] for _ in index_na]

In [7]:
new_lexicon =zip(lexes,lexna)

In [8]:
new_lexicon=list(new_lexicon)

In [9]:
for _ in new_lexicon:
    print(_[0],_[1])

cebolla xonaca
ajo axox
kilo ki
aquí nican
haba ahuax
dinero tomin
chile amo
cucharada chil
amar xumalli
año tlazohtla
sal xihuitl
2 izta
luego ome
8 niman
5 chicuey
noche macuil
elote yohual
juan elo
vender juan
pedernal tlanamaca
6 tecpa
diego chicuacen
4 diego
llorar nahui
comal choca
vivir comal
manteca nemi
hongo chiyahuizotl
flor nanacatl
nopal xochi
escudo nohpal
día chimal
pollo tonal
atole poyox
zanahoria atol
ahora zanahoria
traficante axcan
san pochteca
trabajar san
morir tequiti
casa miqui
cuerpo chan
carne nacayo
masa naca
zapatista textli
maría zapatista
carlos maria
arroz carlo
azúcar arroz
después tzopel
mañana zatepan
ropa moztla
mercado tzotzoma
mucho tianquiz
mexicano miac
tamal mexica
mes tamalli
guerra metz
enojar yao
rico cualan
salir tlatquihua
obispo quiza
viento obispo
ciudad eheca
dialogar altepe
tortilla onotzalli
camisa tlaxcal
hoja huipil
mi izhua
llanto choqui
cuando ihcuac
otro occe
nuevo yancui
dios dios
naranja naranja
frase tlahtol
pasar pano
corazón y

In [10]:
#es[es[0] == "hongo"][0]

In [11]:
es_dummy = es.drop(es.columns[0],axis=1)
na_dummy = na.drop(na.columns[0],axis=1)
es_vectores1 = np.array(es_dummy)
na_vectores1 = np.array(na_dummy)
print(na_vectores1.shape)

(1904, 128)


In [12]:
#es_vectores = [es.iloc[_].loc[1::] for _ in range(es.shape[0])]

In [13]:
print(es.iloc[2645][0])

haba


In [14]:
# Palabra y vector
#indice = 1756
#print(es.iloc[2645][0])
test_vectors = np.array([np.array(es.iloc[indice][1::]).astype(np.float64) for indice in index_es])
#test_vectors = np.array(es.iloc[2645][1::]).astype(np.float64)
#test_vectors.resize(1,128)
print(test_vectors.shape)

(540, 128)


In [15]:
sess = tf.Session()

In [16]:
saver = tf.train.import_meta_graph('./models/Adagrad_H_305_LR_0.433.ckpt.meta')
saver.restore(sess,tf.train.latest_checkpoint('./models/'))

INFO:tensorflow:Restoring parameters from ./models/Adagrad_H_305_LR_0.433.ckpt


In [17]:
graph = tf.get_default_graph()

In [18]:
#print(sess.run("output/W:0"))

In [19]:
output_W = graph.get_tensor_by_name("output/W:0")
output_b = graph.get_tensor_by_name("output/b:0")
output_xwb = graph.get_tensor_by_name("output/xw_plus_b:0")

In [20]:
X = graph.get_tensor_by_name("input/input_es:0")
y = graph.get_tensor_by_name("input/target_na:0")

In [21]:
print(X.shape)

(?, 128)


In [22]:
feed_dict = {X:test_vectors}

In [23]:
output_NN = graph.get_tensor_by_name("output_1:0")

In [24]:
pred = sess.run(output_NN,feed_dict)
print (type(pred[0]),pred.shape)

<class 'numpy.ndarray'> (540, 128)


In [25]:
#print(pred[0].shape)

In [26]:
top_10=[utils.get_top10_closest(pred[_],na_vectores1) for _ in range(pred.shape[0])]
#top_10=utils.get_top10_closest(pred[0],na_vectores1)

In [27]:
closest = [utils.get_closest_words_to(top_10[_],na) for _ in range(pred.shape[0])]
#closest = utils.get_closest_words_to(top_10,na)

In [28]:
closest[1]

['axox',
 'tlan',
 'diego',
 'dral',
 'federal',
 'castillo',
 'club',
 'rosario',
 'secretario',
 'acamapich']

In [39]:
acc1=[(_,lexna[_] == closest[_][0]) for _ in range(pred.shape[0])]
acc=[(lexna[_] == closest[_][0]) for _ in range(pred.shape[0])]

In [40]:
acc

[True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [43]:
precision_list = (sess.run(tf.cast(acc,tf.int32)))
precision_list

array([1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1,

In [44]:
from collections import Counter

In [45]:
precision = Counter(precision_list)

In [46]:
precision[1]/len(precision_list)

0.9462962962962963